In [119]:
from sec_api import QueryApi, ExtractorApi
import json
import requests
import pandas as pd
from IPython.display import display, HTML
import time

queryAPI = QueryApi(api_key='041c38e656c459b1b336a7bea7d83c057482f669d2e663a1b6fa8864b2071550')
extractorApi = ExtractorApi(api_key='041c38e656c459b1b336a7bea7d83c057482f669d2e663a1b6fa8864b2071550')

In [110]:
#formType:(“10-K”, “10-KT”, “10KSB”, “10KT405”, “10KSB40”, “10-K405”)
class SEC_QUERY:

    def __init__(self, form_type, ticker, size):
        self.form_type = form_type
        self.ticker = ticker
        self.size = size
        self.query = {
            "query": { 
                "query_string" : {
                    "query": f"formType:\"{self.form_type}\" AND ticker:{self.ticker}",
                }
            },
            "from": "0",
            "size": size,
            "sort": [{"filedAt": {"order": "desc"}}]
        }
    
    def get_response(self, index:int):
        response = queryAPI.get_filings(self.query)
        return json.dumps(response["filings"][index], indent=2)
    
    def get_filing(self):
        return queryAPI.get_filings(self.query)

    def extract(self, index:int):
        query_str = self.get_response(index)
        q_dict = json.loads(query_str)
        filing_url = q_dict['linkToHtml']

        xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"
        api_key = '041c38e656c459b1b336a7bea7d83c057482f669d2e663a1b6fa8864b2071550'

        final_url = xbrl_converter_api_endpoint + "?htm-url=" + filing_url + "&token=" + api_key
        response = requests.get(final_url)

        final_json = json.loads(response.text)
        return final_json, filing_url, q_dict

10-K supported items [1, 1A, 1B, 2, 3, 4, 5, 6, 7, 7A, 8, 9, 9A, 9B, 10, 11, 12, 13, 14, 15]

In [111]:
query1 = SEC_QUERY("10-K", "TSLA", "10")
init_query, f_url, query_response = query1.extract(0)

#section_text = extractorApi.get_section(f_url, "8", "text")
#section_html = extractorApi.get_section(f_url, "8", 'html')

In [100]:
csv = init_query['StatementsOfIncome']
columns = list(csv.keys())
values = []

for entry_list in csv.values():
    entry_values = [entry.get('value', None) for entry in entry_list]
    values.append(entry_values)

values_transposed = list(map(list, zip(*values)))

df_income = pd.DataFrame(values_transposed, columns=columns)
df_income

,RevenueFromContractWithCustomerExcludingAssessedTax,CostOfRevenue,GrossProfit,ResearchAndDevelopmentExpense,SellingGeneralAndAdministrativeExpense,RestructuringAndOtherExpenses,OperatingExpenses,OperatingIncomeLoss,InvestmentIncomeInterest,InterestExpense,OtherNonoperatingIncomeExpense,IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,IncomeTaxExpenseBenefit,ProfitLoss,NetIncomeLossAttributableToNoncontrollingInterest,NetIncomeLoss,EarningsPerShareBasic,EarningsPerShareDiluted,WeightedAverageNumberOfSharesOutstandingBasic,WeightedAverageNumberOfDilutedSharesOutstanding
0,78509000000,65121000000,17660000000,3969000000,4800000000,0,8769000000,8891000000,1066000000,156000000,172000000,9973000000,-5001000000,14974000000,-23000000,14997000000,4.73,4.30,3174000000,3485000000
1,67210000000,49599000000,20853000000,3075000000,3946000000,176000000,7197000000,13656000000,297000000,191000000,-43000000,13719000000,1132000000,12587000000,31000000,12556000000,4.02,3.62,3130000000,3475000000
2,44125000000,32415000000,13606000000,2593000000,4517000000,-27000000,7083000000,6523000000,56000000,371000000,135000000,6343000000,699000000,5644000000,125000000,5519000000,1.87,1.63,2959000000,3386000000


In [101]:
def get_statement_of_income(final_json):
    income_statement_store = {}

    for usGAAP in final_json['StatementsOfIncome']:
        values = []
        indicies = []

        for fact in final_json['StatementsOfIncome'][usGAAP]:
            if 'segment' not in fact:
                index = fact['period']['startDate'] +'-'+fact['period']['endDate']
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)
        
        income_statement_store[usGAAP] = pd.Series(values, index=indicies)
    income_statement = pd.DataFrame(income_statement_store)
    return income_statement.T

In [102]:
get_statement_of_income(init_query)

,2023-01-01-2023-12-31,2022-01-01-2022-12-31,2021-01-01-2021-12-31
RevenueFromContractWithCustomerExcludingAssessedTax,96773000000,81462000000,53823000000
CostOfRevenue,79113000000,60609000000,40217000000
GrossProfit,17660000000,20853000000,13606000000
ResearchAndDevelopmentExpense,3969000000,3075000000,2593000000
SellingGeneralAndAdministrativeExpense,4800000000,3946000000,4517000000
RestructuringAndOtherExpenses,0,176000000,-27000000
OperatingExpenses,8769000000,7197000000,7083000000
OperatingIncomeLoss,8891000000,13656000000,6523000000
InvestmentIncomeInterest,1066000000,297000000,56000000
InterestExpense,156000000,191000000,371000000


In [103]:
def get_balance_sheet(final_json):
    balance_sheet_store = {}
    for usGAAP in final_json['BalanceSheets']:
        values = []
        indicies = []

        for fact in final_json['BalanceSheets'][usGAAP]:
            if 'segment' not in fact:
                index = fact['period']['instant']

                if index in indicies:
                    continue
                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])
                indicies.append(index)
            
            balance_sheet_store[usGAAP] = pd.Series(values, index=indicies)
        
    balance_sheet = pd.DataFrame(balance_sheet_store)
    return balance_sheet.T

In [104]:
get_balance_sheet(init_query)

,2021-12-31,2022-12-31,2023-12-31
CashAndCashEquivalentsAtCarryingValue,17576000000,16253000000,16398000000
ShortTermInvestments,NaN,5932000000,12696000000
AccountsReceivableNetCurrent,NaN,2952000000,3508000000
InventoryNet,NaN,12839000000,13626000000
PrepaidExpenseAndOtherAssetsCurrent,NaN,2941000000,3388000000
AssetsCurrent,NaN,40917000000,49616000000
DeferredCostsLeasingNetNoncurrent,NaN,NaN,NaN
LeasedAssetsNet,NaN,NaN,NaN
PropertyPlantAndEquipmentNet,NaN,23548000000,29725000000
OperatingLeaseRightOfUseAsset,NaN,2563000000,4180000000


In [118]:
query_file = query1.get_filing(0)
acc_num = []

for filing in query_file['filings']:
    acc_num.append(filing['accessionNo'])


['0001628280-24-002390',
 '0000950170-23-001409',
 '0001564590-22-016871',
 '0000950170-22-000796',
 '0001564590-21-022604',
 '0001564590-21-004599',
 '0001564590-20-018984',
 '0001564590-20-004475',
 '0001564590-19-003165',
 '0001564590-18-002956']

In [76]:
#Making output more readable 
def pprint(text, line_length=100):
    words = text.split(' ')
    lines = []
    current_line = ' '
    for word in words:
        if len(current_line + ' '+ word) <= line_length:
            current_line += ' '+word
        else:
            lines.append(current_line.strip())
            current_line = word
    if current_line:
        lines.append(current_line.strip())
    print('\n'.join(lines))

In [77]:
pprint(section_text[0:3000])

ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA 

Index to Consolidated Financial Statements
##TABLE_START Page 

Report of Independent Registered Public Accounting Firm (PCAOB ID: 238 )
Consolidated Balance Sheets 

Consolidated Statements of Operations 

Consolidated Statements of
Comprehensive Income 

Consolidated Statements of Redeemable Noncontrolling Interests and Equity
Consolidated Statements of Cash Flows 

Notes to Consolidated Financial Statements
##TABLE_END

Report of Independent Registered Public Accounting Firm 

To the Board of Directors
and Stockholders of Tesla, Inc. 

Opinions on the Financial Statements and Internal Control over
Financial Reporting 

We have audited the accompanying consolidated balance sheets of Tesla, Inc.
and its subsidiaries (the &#8220;Company&#8221;) as of December 31, 2023 and 2022, and the related
consolidated statements of operations, of comprehensive income, of redeemable noncontrolling
interests and equity and of cash flows for each 

In [ ]:
display(HTML(section_html[0:150000]))